firebase rpc wrapper?


In [ ]:
var cors = require('cors')({origin: true});
var process = require('process');
var functions = require('firebase-functions');
var admin = require('firebase-admin');
var importer = require('../Core');
var getResult = importer.import('rpc result');

try {
    admin.initializeApp(functions.config().firebase);
} catch (e) {
    console.log(e);
}

if(typeof module.exports === 'undefined') {
    module.exports = {};
}

module.exports.rpc = functions.https.onRequest((req, res) => {
    process.env.HOME = process.env.HOMEPATH = process.env.USERPROFILE = '';
    const config = functions.config();
    const services = Object.keys(config).reduce(k => {
        Object.keys(config[k]).forEach(s => {
            process.env[k.toUpperCase() + '_' + s.toUpperCase()] = config[k][s];
        });
    });
    
    cors(req, res, () => {
        return Promise.resolve([])
            .then(() => getResult({
                command: req.body['function'] || req.query['function'],
                result: importer.interpret(req.body['function'] || req.query['function']),
                body: req.method === 'POST' ? req.body : req.query,
                circles: ['Public']
            }))
            .then(r => {
                const result = Object.getOwnPropertyNames(r).reduce((alt, key) => {
                    alt[key] = r[key] + '';
                    return alt;
                }, {});
                console.error(result);
                return res.status(200).send(r);
            })
            // TODO: object assign error?
            .catch(e => { 
                const result = Object.getOwnPropertyNames(e).reduce((alt, key) => {
                    alt[key] = e[key] + '';
                    return alt;
                }, {});
                console.error(result);
                return res.status(500).send(result);
            });
    });
})


google cloud function rpc wrapper?


In [ ]:
var cookie = require('cookie');
var cors = require('cors')({origin: true});
var importer = require('../Core');
var getResult = importer.import('rpc result');
var getEnvironment = importer.import('get environment');

var expressToResult = (req, res) => getResult({
    command: (req.body || {})['function'] || (req.query || {})['function'],
    result: importer.interpret((req.body || {})['function'] || (req.query || {})['function']),
    body: req.method === 'POST' ? req.body : req.query,
    circles: ['Public']
})

var logResult = (r) => {
    // TODO: call storeResult()?
    var result = typeof r === 'object'
        ? Object.keys(r).reduce((alt, key) => {
            alt[key] = r[key] + '';
            return alt;
        }, {})
        : ('' + r);
    console.log(result);
    return result;
}

function expressHandler(req, res) {
    getEnvironment();
    req.cookies = cookie.parse(req.get('cookie') || '');
    res.setHeader('Cache-Control', 'private');
    cors(req, res, () => {});
    return Promise.resolve()
        .then(() => expressToResult(req, res))
        .then(r => (res.status(200), r))
        .catch(e => (res.status(500), e))
        .then(r => logResult(r))
        .then(r => res.send(r));
}

module.exports.handler = expressHandler;



test google express rpc handler?


In [ ]:
var importer = require('../Core');
var {handler} = importer.import('google cloud function rpc wrapper');

if(typeof $$ !== 'undefined') {
    $$.async();
    handler({
        headers: {origin: 'google.com'},
        get: r => null,
        query: {
            function: 'create a copy of study sauce template',
            email: 'megamindbrian@gmail.com'
        }
    }, {
        getHeader: h => {},
        setHeader: h => console.log(`header ${h}`),
        status: s => console.log(`status ${s}`),
        send: r => console.log(r)
    })
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


generic gcloud function handler?



In [ ]:
var importer = require('../Core')
var {
    niceName, getExports, getParameters, selectAst
} = importer.import(['nice name', 'get exports', 'get parameters', 'select ast'])

async function handler(req, res) {
    var func = require('./entry.js')
    var parameters;
    if(!req && process.stdout.isTTY) {
        req = {query: {}}
        Array.from(process.argv).forEach(arg => {
            req.query[arg.split('=')[0]] = arg.split('=').slice(1).join('=')
        })
        
    }
    parameters = Object.values(req.query)
        .concat(Object.values(req.body))
        .concat(Object.values(req.form))
    res.set('Access-Control-Allow-Origin', '*')
    return await func(...parameters)
        .then(r => !res ? console.log(r) : res.status(200).send(r))
        .catch(e => !res ? console.log(e) : res.status(500).send(e))
}

// use syntax to alter the function handler above
//  to pull out parameters and call the requested function
//  slightly simpler code than including the entire library
function transformHandler(entry) {
    if(!Array.isArray(entry)) {
        entry = [entry]
    }
    var interpret = importer.interpret(entry)
    interpret.forEach(cell => {
        var exports = getExports(cell.code)
        var func = selectAst(`
//FunctionDeclaration[contains(@name, "${exports[0]}")]`,
                                 cell.code)
        var parameters = getParameters(func)
        
        // replace the require statement
        // TODO: make this part generic, moving vars to parameters
        var doc = handler.toString()
        var requireStmt = selectAst(`
//VariableDeclaration[./*/*[@name = "func"]]/AssignmentOperator`, doc)
        requireStmt.replace(`require('./${niceName(cell)}')`)
        requireStmt.replace(func)
                
        // replace function call with new named
        var callStmt = selectAst(`
//CallExpression[@name = "func"]`, doc)
        callStmt.replace(exports[0])

        // replace the parameters with names
        var replaceParams = parameters
            .map(p => `req.body['${p}'] || req.form['${p}'] || req.query['${p}']`)
            .join(',\n')
        
        var paramsStmt = selectAst([`//AssigmentOperator[@name = "parameters"]`], doc)
        params.replace(`[${replaceParams}]`)
    })
    // create fake cells pointing to here for exporting or compiling elsewhere
}

module.exports = {
    transformHandler,
    handler
}
